<a href="https://colab.research.google.com/github/amiranissian/ai-ml-principles-exercises/blob/main/ML-training/logging-trainings-with-WandB/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [10]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [2]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [3]:
def get_mnist_data():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [11]:
batch_size = 128
epochs = 5

In [12]:
# Get the training data
x_train, y_train = get_mnist_data()[0]

In [13]:
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1)

Epoch 1/5
422/422 [==============================] - 42s 99ms/step - loss: 0.3012 - accuracy: 0.9159 - val_loss: 0.0756 - val_accuracy: 0.9788
Epoch 2/5
422/422 [==============================] - 41s 98ms/step - loss: 0.0796 - accuracy: 0.9755 - val_loss: 0.0615 - val_accuracy: 0.9825
Epoch 3/5
422/422 [==============================] - 41s 97ms/step - loss: 0.0604 - accuracy: 0.9816 - val_loss: 0.0504 - val_accuracy: 0.9847
Epoch 4/5
422/422 [==============================] - 40s 96ms/step - loss: 0.0475 - accuracy: 0.9851 - val_loss: 0.0491 - val_accuracy: 0.9875
Epoch 5/5
422/422 [==============================] - 44s 103ms/step - loss: 0.0406 - accuracy: 0.9874 - val_loss: 0.0480 - val_accuracy: 0.9862


In [16]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 8.0 MB/s 
     |████████████████████████████████| 182 kB 68.2 MB/s 
     |████████████████████████████████| 168 kB 59.8 MB/s 
     |████████████████████████████████| 62 kB 896 kB/s 
     |████████████████████████████████| 168 kB 69.2 MB/s 
     |████████████████████████████████| 166 kB 75.4 MB/s 
     |████████████████████████████████| 166 kB 71.9 MB/s 
     |████████████████████████████████| 162 kB 61.3 MB/s 
     |████████████████████████████████| 162 kB 72.3 MB/s 
     |████████████████████████████████| 158 kB 73.4 MB/s 
     |████████████████████████████████| 157 kB 72.8 MB/s 
     |████████████████████████████████| 157 kB 53.4 MB/s 
     |████████████████████████████████| 157 kB 63.4 MB/s 
     |████████████████████████████████| 157 kB 75.9 MB/s 
     |████████████████████████████████| 157 kB 81.0 MB/s 
     |████████████████████████████

In [17]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [22]:
batch_size = 128
epochs = 15
wandb.init(project="ai-ml-exercise", config={"batch_size": batch_size, "epochs": epochs})

In [23]:
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])
wandb.finish()

Epoch 1/15
422/422 [==============================] - ETA: 0s - loss: 0.0149 - accuracy: 0.9954

wandb: Adding directory to artifact (/content/wandb/run-20221206_134824-2xqsllln/files/model-best)... Done. 0.0s


422/422 [==============================] - 43s 103ms/step - loss: 0.0149 - accuracy: 0.9954 - val_loss: 0.0453 - val_accuracy: 0.9888
Epoch 2/15
422/422 [==============================] - ETA: 0s - loss: 0.0120 - accuracy: 0.9962

wandb: Adding directory to artifact (/content/wandb/run-20221206_134824-2xqsllln/files/model-best)... Done. 0.0s


422/422 [==============================] - 41s 98ms/step - loss: 0.0120 - accuracy: 0.9962 - val_loss: 0.0416 - val_accuracy: 0.9898
Epoch 3/15
422/422 [==============================] - 40s 96ms/step - loss: 0.0108 - accuracy: 0.9967 - val_loss: 0.0467 - val_accuracy: 0.9890
Epoch 4/15
422/422 [==============================] - 40s 95ms/step - loss: 0.0099 - accuracy: 0.9968 - val_loss: 0.0426 - val_accuracy: 0.9902
Epoch 5/15
422/422 [==============================] - 40s 95ms/step - loss: 0.0063 - accuracy: 0.9982 - val_loss: 0.0443 - val_accuracy: 0.9910
Epoch 6/15
422/422 [==============================] - 40s 95ms/step - loss: 0.0065 - accuracy: 0.9980 - val_loss: 0.0501 - val_accuracy: 0.9885
Epoch 7/15
422/422 [==============================] - 42s 99ms/step - loss: 0.0066 - accuracy: 0.9979 - val_loss: 0.0524 - val_accuracy: 0.9890
Epoch 8/15
422/422 [==============================] - 40s 95ms/step - loss: 0.0055 - accuracy: 0.9983 - val_loss: 0.0474 - val_accuracy: 0.9900
Epo

accuracy,▁▃▄▄▆▆▆▆▇▇▇▇▇██
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▆▅▃▃▃▃▂▂▂▂▂▁▁
val_accuracy,▂▅▂▆█▁▂▅▁▃▄▅▆▅█
val_loss,▃▁▃▁▂▄▅▃█▆▇▆▅█▅
accuracy,0.99898
best_epoch,1
best_val_loss,0.0416
epoch,14
loss,0.00344
val_accuracy,0.991
